### Controlla se il link passato in ingresso appartiene alla whitelist

In [ ]:
def checkWhitelist(link):
    with open('whitelist.txt', 'r') as f:
        whitelist = [line.strip() for line in f]
    if link in whitelist:
        return True
    else:
        return False

### Controlla se il link passato in ingresso appartiene a una blacklist

In [ ]:
def checkBlacklist(link):
    with open('blackList.txt', 'r') as f:
        blacklist = [line.strip() for line in f]
    if link in blacklist:
        return True
    else:
        return False

### Controlla se il link passato in ingresso appartiene a una cache memorizzata all'interno del modulo

In [ ]:
def checkLocalList(link, local_list):
    if link in local_list:
        return True
    else:
        return False

### Verifica del certificato del link

In [ ]:
import ssl
import socket

class CertificateControl:
    def __init__(self):
        # Lista di CA note per offrire certificati gratuiti
        # Nota: Questa lista va mantenuta aggiornata.
        self.free_cas = [
            "Let's Encrypt",
            "ZeroSSL",
            "Cloudflare",
            "cPanel",
            "GoGetSSL",
            "Buypass"
        ]

    def analyze(self, hostname, port=443):
        context = ssl.create_default_context()
        try:
            with socket.create_connection((hostname, port), timeout=5) as sock:
                with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                    cert = ssock.getpeercert()
                    if not cert:
                        return {"error": "Nessun certificato trovato o validazione fallita"}

                    def get_value(field_list, key_name):
                        for item in field_list:
                            for sub_item in item:
                                if sub_item[0] == key_name:
                                    return sub_item[1]
                        return None

                    issuer_org = get_value(cert.get('issuer', []), 'organizationName')
                    issuer_cn = get_value(cert.get('issuer', []), 'commonName')
                    subject_cn = get_value(cert.get('subject', []), 'commonName')

                    print(f"--- Analisi per {hostname} ---")
                    print(f"Emittente (Issuer): {issuer_org} ({issuer_cn})")

                    is_self_signed = (issuer_cn == subject_cn) and (issuer_org == get_value(cert.get('subject', []), 'organizationName'))

                    if is_self_signed:
                        return {"status": "WARNING", "reason": "Certificato Self-Signed (Autofirmato)"}
                        

                    if issuer_org:
                        for free_ca in self.free_cas:
                            if free_ca.lower() in issuer_org.lower():
                                return {"status": "WARNING", "reason": f"Certificato emesso da ente gratuito: {issuer_org}"}

                    return {"status": "OK", "reason": "Certificato standard/a pagamento", "issuer": issuer_org}

        except ssl.SSLCertVerificationError as e:
            return {"status": "DANGER", "reason": f"Verifica SSL fallita (probabile Self-Signed non trustato): {e.verify_message}"}
        except Exception as e:
            return {"status": "ERROR", "reason": str(e)}

# --- ESEMPI DI UTILIZZO ---

cert_analyzer = CertificateControl()

# Esempio 1: Sito con Let's Encrypt (Gratuito)
print(cert_analyzer.analyze("stackoverflow.com")) 

# Esempio 2: Sito con certificato aziendale (spesso DigiCert, Entrust, etc.)
print(cert_analyzer.analyze("www.google.com"))

# Esempio 3: Test su un sito self-signed (se ne hai uno interno)
# print(cert_analyzer.analyze("tuo-server-interno"))



--- Analisi per stackoverflow.com ---
Emittente (Issuer): Let's Encrypt (E8)
{'status': 'WARNING', 'reason': "Certificato emesso da ente gratuito: Let's Encrypt"}
--- Analisi per www.google.com ---
Emittente (Issuer): Google Trust Services (WR2)
{'status': 'OK', 'reason': 'Certificato standard/a pagamento', 'issuer': 'Google Trust Services'}


### Verifica se il link è scritto in maniera sospetta

In [45]:
import re
from difflib import SequenceMatcher
from urllib.parse import urlparse

class UrlValidator:
    def __init__(self):
        # 1. Regex Struttura
        self.structure_regex = re.compile(
            r'^(https?://)' 
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+)' 
            r'([A-Z]{2,63})' 
            r'(?:/?|[/?]\S+)$', 
            re.IGNORECASE
        )
        
        # 2. Regex Anti-Omografia (Cerca caratteri Unicode nascosti)
        self.homograph_regex = re.compile(r'[^\x00-\x7F]')
        
        # 3. Lista VIP per Typosquatting (Esempio)
        #self.vip_domains = ['paypal.com', 'google.com', 'apple.com', 'microsoft.com']

    def analyze(self, url):
        report = {'valid_syntax': True, 'risk_flags': []}

        # Check 1: Sintassi
        if not self.structure_regex.match(url):
            report['valid_syntax'] = False
            report['risk_flags'].append("Formato URL non valido")
            return report # Inutile continuare se non è un URL

        # Check 2: Omografia (Unicode)
        if self.homograph_regex.search(url):
            # Nota: Punycode (xn--) è un altro indicatore di IDN
            report['risk_flags'].append("Rilevati caratteri non-ASCII (Possibile attacco Omografico)")

        # Check 3: Typosquatting
        # Estraiamo solo il dominio puro (es. 'g0ogle.com')
        '''try:
            domain_part = urlparse(url).netloc
            for safe_domain in self.vip_domains:
                if domain_part == safe_domain:
                    continue # È il sito vero
                
                # Calcolo similarità visiva
                ratio = SequenceMatcher(None, domain_part, safe_domain).ratio()
                if 0.80 < ratio < 1.0:
                    report['risk_flags'].append(f"Sospetto Typosquatting di '{safe_domain}' ({ratio:.0%})")
        except:
            pass
        '''
        return report

# --- TEST ---
if __name__ == "__main__":
    validator = UrlValidator()
    
    urls = [
        "https://www.google.com",       # Ok
        "https://dhl-dellver.com",       # Typosquatting
        "https://www.googIe.com",       # Typosquatting (I maiuscola al posto di l)
        "https://www.xn--pple-43d.com", # Punycode / Omografico
        "ftp://sito-strano"             # Sintassi errata (manca TLD o protocollo errato)
    ]
    
    for u in urls:
        print(f"Analisi: {u}")
        print(validator.analyze(u))
        print("-" * 30)

Analisi: https://www.google.com
{'valid_syntax': True, 'risk_flags': []}
------------------------------
Analisi: https://dhl-dellver.com
{'valid_syntax': True, 'risk_flags': []}
------------------------------
Analisi: https://www.googIe.com
{'valid_syntax': True, 'risk_flags': []}
------------------------------
Analisi: https://www.xn--pple-43d.com
{'valid_syntax': True, 'risk_flags': []}
------------------------------
Analisi: ftp://sito-strano
{'valid_syntax': False, 'risk_flags': ['Formato URL non valido']}
------------------------------


### Cerca all'interno di banche dati di phishing se il sito web appare

#### Phishing.army

In [32]:
import requests
import os
from urllib.parse import urlparse

class PhishingArmyControl:
    def __init__(self, db_folder='.'):
        self.PA_URL = 'https://phishing.army/download/phishing_army_blocklist_extended.txt'
        self.DB_FILE = os.path.join(db_folder, 'phishing_army.txt')
        
       
        self.blocked_domains = set()

    def _get_domain(self, url):
        """
        Estrae il dominio principale da un URL.
        Es: 'https://bad.site.com/login' -> 'bad.site.com'
        """
        if not url: return ""
        
        # Aggiunge schema se manca per far funzionare urlparse
        if not url.startswith(('http://', 'https://')):
            url = 'http://' + url
            
        try:
            parsed = urlparse(url)
            # Rimuove la porta se presente (es. :8080)
            return parsed.netloc.split(':')[0]
        except Exception:
            return ""

    def _download_db(self):
        """Scarica la lista aggiornata."""
        print("🔄 [Phishing Army] Scaricamento aggiornamenti...")
        try:
            response = requests.get(self.PA_URL, stream=True)
            if response.status_code == 200:
                with open(self.DB_FILE, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print("✅ [Phishing Army] Lista aggiornata.")
                return True
            else:
                print(f"❌ [Phishing Army] Errore download: {response.status_code}")
                return False
        except Exception as e:
            print(f"❌ [Phishing Army] Errore connessione: {e}")
            return False

    def load_data(self, force_update=False):
        """Carica la lista in memoria RAM."""
        # Scarica se non esiste o se richiesto
        if force_update or not os.path.exists(self.DB_FILE):
            success = self._download_db()
            if not success and not os.path.exists(self.DB_FILE):
                return

        print("📂 [Phishing Army] Caricamento domini in memoria...")
        try:
            with open(self.DB_FILE, 'r', encoding='utf-8') as f:
                # Set Comprehension: Legge, pulisce spazi, ignora commenti (#)
                self.blocked_domains = {
                    line.strip() 
                    for line in f 
                    if line.strip() and not line.startswith('#')
                }
            print(f"🔹 [Phishing Army] {len(self.blocked_domains)} domini caricati.")
        except Exception as e:
            print(f"❌ [Phishing Army] Errore lettura file: {e}")

    def check_url(self, url):
        """
        Controlla se il DOMINIO dell'URL è nella blocklist.
        """
        if not self.blocked_domains:
            print("⚠️ [Phishing Army] DB vuoto. Esegui load_data().")
            return None

        target_domain = self._get_domain(url)
        
        if target_domain in self.blocked_domains:
            return {
                'detected': True,
                'source': 'Phishing Army',
                'domain_matched': target_domain,
                'details': 'Domain listed in Blocklist Extended'
            }
        
        return None

#### PhishTank

In [33]:
import requests
import json
import gzip
import shutil
import os

class PhishTankControl:
    def __init__(self, db_folder='.'):
        """
        Inizializza il gestore PhishTank.
        :param db_folder: Cartella dove salvare i file JSON (default: cartella corrente)
        """
        self.PT_URL = 'http://data.phishtank.com/data/online-valid.json.gz'
        self.GZ_FILE = os.path.join(db_folder, 'phishtank.json.gz')
        self.JSON_FILE = os.path.join(db_folder, 'phishtank.json')
        
        # Database in memoria (Dizionario: URL -> Target)
        self.database = {}
        
        # Header necessario per bypassare controlli di sicurezza base
        self.headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

    def _download_and_extract(self):
        """Metodo interno per scaricare e decomprimere il database."""
        print("🔄 [PhishTank] Scaricamento aggiornamenti...")
        try:
            # 1. Download
            response = requests.get(self.PT_URL, headers=self.headers, stream=True)
            if response.status_code == 200:
                with open(self.GZ_FILE, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                # 2. Decompressione
                print("📦 [PhishTank] Decompressione database...")
                with gzip.open(self.GZ_FILE, 'rb') as f_in:
                    with open(self.JSON_FILE, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                
                print("✅ [PhishTank] Aggiornamento completato.")
                return True
            else:
                print(f"❌ [PhishTank] Errore download: Status {response.status_code}")
                return False
        except Exception as e:
            print(f"❌ [PhishTank] Errore critico: {e}")
            return False

    def load_data(self, force_update=False):
        """
        Carica i dati in RAM. Scarica se il file non esiste o se force_update è True.
        Filtra solo i risultati 'verified' == 'yes'.
        """
        if force_update or not os.path.exists(self.JSON_FILE):
            success = self._download_and_extract()
            if not success and not os.path.exists(self.JSON_FILE):
                print("⚠️ [PhishTank] Impossibile caricare i dati.")
                return

        print("📂 [PhishTank] Caricamento dati in memoria (solo verificati)...")
        try:
            with open(self.JSON_FILE, 'r', encoding='utf-8') as f:
                raw_data = json.load(f)
            
            # --- IL FILTRO RICHIESTO ---
            # Carichiamo solo se verified == 'yes'
            # Salviamo url -> target per risparmiare RAM
            self.database = {
                entry['url']: entry['target']
                for entry in raw_data
                #if entry.get('verified') == 'yes'
            }
            
            count = len(self.database)
            print(f"🔹 [PhishTank] {count} URL verificati pronti all'uso.")
            
        except Exception as e:
            print(f"❌ [PhishTank] Errore lettura JSON: {e}")

    def check_url(self, url):
        """
        Controlla se un URL è nel database.
        Restituisce un dizionario con i dettagli o None.
        """
        if not self.database:
            print("⚠️ [PhishTank] Database vuoto. Esegui load_data() prima.")
            return None

        # Ricerca diretta 
        target = self.database.get(url)
        
        if target:
            return {
                'detected': True,
                'source': 'PhishTank',
                'target': target,
                'verified': True 
            }
        
        return None

### Analisi reputazione VirusTotal

In [34]:
import requests
import base64
import json
import time

class VirusTotalControl:
    def __init__(self, api_key):
        """
        Inizializza il gestore VirusTotal.
        :param api_key: La tua chiave API privata (stringa).
        """
        self.api_key = api_key
        self.base_url = "https://www.virustotal.com/api/v3"
        self.headers = {
            "x-apikey": self.api_key
        }

    def _url_to_id(self, url):
        """
        Converte un URL nell'identificatore richiesto da VirusTotal API v3.
        """
        try:
            url_bytes = url.encode('utf-8')
            base64_bytes = base64.urlsafe_b64encode(url_bytes)
            base64_str = base64_bytes.decode('utf-8')
            return base64_str.strip("=")
        except Exception as e:
            print(f"❌ [VirusTotal] Errore codifica URL: {e}")
            return None

    def check_url(self, url):
        """
        Controlla la reputazione di un URL.
        Restituisce un dizionario con i risultati o None se errore/quota superata.
        """
        url_id = self._url_to_id(url)
        if not url_id:
            return None

        endpoint = f"{self.base_url}/urls/{url_id}"

        try:
            response = requests.get(endpoint, headers=self.headers)

            if response.status_code == 200:
                data = response.json()
                attributes = data['data']['attributes']
                stats = attributes['last_analysis_stats']
                
                # Calcolo del punteggio di rischio
                malicious_count = stats.get('malicious', 0)
                suspicious_count = stats.get('suspicious', 0)
                
                is_flagged = (malicious_count + suspicious_count) > 0
                
                return {
                    'detected': is_flagged,
                    'source': 'VirusTotal',
                    'malicious_votes': malicious_count,
                    'suspicious_votes': suspicious_count,
                    'total_votes': sum(stats.values()),
                    'permalink': attributes.get('last_analysis_results') 
                }
            
            elif response.status_code == 404:
                # 404 in VT v3 significa "URL mai analizzato prima"
                return {'detected': False, 'source': 'VirusTotal', 'status': 'Unknown/New'}
            
            elif response.status_code == 429:
                print("⚠️ [VirusTotal] Quota API superata (4 richieste/minuto).")
                return None
                
            elif response.status_code == 401:
                print("❌ [VirusTotal] API Key non valida.")
                return None
            
            else:
                print(f"❌ [VirusTotal] Errore HTTP: {response.status_code}")
                return None

        except Exception as e:
            print(f"❌ [VirusTotal] Errore connessione: {e}")
            return None
        



In [41]:
import os
import sys
import time
from dotenv import load_dotenv

# Import dei 3 moduli
#from phishtank_manager import PhishTankManager
#from phishing_army_manager import PhishingArmyManager  # <--- NUOVO
#from virustotal_manager import VirusTotalManager

# --- CONFIGURAZIONE ---
load_dotenv()
VT_API_KEY = os.getenv('VIRUSTOTAL_APIKEY')

if not VT_API_KEY:
    print("❌ ERRORE: Chiave VirusTotal mancante nel file .env")
    sys.exit(1)

# --- INIZIALIZZAZIONE ---
print("\n⚙️  AVVIO MOTORI DI SICUREZZA...\n")

# 1. Motori Locali (Veloci)
phish_tank = PhishTankControl()
phish_tank.load_data()

phish_army = PhishingArmyControl() # <--- NUOVO
phish_army.load_data()             # <--- NUOVO

# 2. Motore Cloud (Lento/Limitato)
vt_engine = VirusTotalControl(VT_API_KEY)

# --- LISTA URL DA TESTARE ---
urls_to_test = [
    "https://www.google.com",                    # Pulito
    "http://interstroy.su/verification",         # Probabile Phishing Army
    "http://www.travels-jordan.com/",            # Probabile PhishTank
    "http://testsafebrowsing.appspot.com/"       # Malevolo generico
]

print("\n🔍 --- INIZIO ANALISI MULTI-LAYER --- 🔍\n")

for url in urls_to_test:
    print(f"Target: {url}")
    is_safe = True
    print(f"Analisi: {url}")
    print(validator.analyze(url))
    print("-" * 30)
    # --- LIVELLO 1: PhishTank (URL Esatto) ---
    if is_safe:
        res_pt = phish_tank.check_url(url)
        if res_pt:
            print(f"   🛑 RILEVATO DA PHISHTANK!")
            print(f"      Target imitato: {res_pt['target']}")
            is_safe = False
    
    # --- LIVELLO 2: Phishing Army (Dominio) ---
    # Eseguiamo questo check solo se PhishTank non ha già trovato nulla
    if is_safe:
        res_pa = phish_army.check_url(url)
        if res_pa:
            print(f"   🛑 RILEVATO DA PHISHING ARMY!")
            print(f"      Dominio bloccato: {res_pa['domain_matched']}")
            is_safe = False
            
    # --- LIVELLO 3: VirusTotal (API Cloud) ---
    # Lo usiamo solo se i controlli locali sono puliti
    if is_safe:
        print("   ☁️  Controllo VirusTotal in corso...")
        res_vt = vt_engine.check_url(url)
        
        if res_vt and res_vt['detected']:
            print(f"   ☣️  RILEVATO DA VIRUSTOTAL!")
            print(f"      Punteggio: {res_vt['malicious_votes']}/{res_vt['total_votes']}")
        elif res_vt:
            print("   ✅ Pulito (VirusTotal).")
        else:
            print("   ⚠️ Errore/Quota VirusTotal.")
            
        # Pausa obbligatoria per API Free (4 richieste/min)
        time.sleep(15)
    
    print("-" * 40)

print("\n✅ Scansione completata.")


⚙️  AVVIO MOTORI DI SICUREZZA...

📂 [PhishTank] Caricamento dati in memoria (solo verificati)...
🔹 [PhishTank] 50416 URL verificati pronti all'uso.
📂 [Phishing Army] Caricamento domini in memoria...
🔹 [Phishing Army] 153716 domini caricati.

🔍 --- INIZIO ANALISI MULTI-LAYER --- 🔍

Target: https://www.google.com
Analisi: https://www.google.com
{'valid_syntax': True, 'risk_flags': ["⚠️ Sospetto Typosquatting di 'google.com' (83%)"]}
------------------------------
   ☁️  Controllo VirusTotal in corso...
   ✅ Pulito (VirusTotal).
----------------------------------------
Target: http://interstroy.su/verification
Analisi: http://interstroy.su/verification
{'valid_syntax': True, 'risk_flags': []}
------------------------------
   ☁️  Controllo VirusTotal in corso...
   ✅ Pulito (VirusTotal).
----------------------------------------
Target: http://www.travels-jordan.com/
Analisi: http://www.travels-jordan.com/
{'valid_syntax': True, 'risk_flags': []}
------------------------------
   ☁️  Cont